In [1]:
import glob
import getopt
import pickle
import os, sys
import numpy as np
from casadi import *
from numpy.lib.utils import info
from scipy.sparse import csc_matrix

sys.path.append('..')

from Common.util import *
from Common.custom_dataclass import *

In [86]:
waypoints = np.loadtxt('../../Data/2D_waypoints.txt')
waypoints = waypoints.tolist()

def nearest_state(curr_state, waypoints, vx_des, vy_des):
    l = len(waypoints)

    x_r = curr_state[0]
    y_r = curr_state[1]

    nearest_state = [0, 0, 0, MX(vx_des), MX(vy_des)]

    # Search nearest waypoint index
    dx = vcat([(x_r - pose[0])**2 for pose in waypoints])
    dy = vcat([(y_r - pose[1])**2 for pose in waypoints])
    dist_sq = dx + dy

    waypoints = MX(hcat(waypoints))
    
    minimum = inf
    idx_1 = 0
    for i in range(l):
        idx_1 = if_else(dist_sq[i] < minimum, i, idx_1)
        minimum = if_else(dist_sq[i] < minimum, dist_sq[i], minimum)

    idx_2 = if_else(dist_sq[idx_1 + 1] < dist_sq[idx_1 - 1], idx_1 + 1, idx_1 - 1)

    x1 = waypoints[0, idx_1]
    y1 = waypoints[1, idx_1]
    x2 = waypoints[0, idx_2]
    y2 = waypoints[1, idx_2]

    l = ((x2 ** 2) + (y2 ** 2) - (x1 * x2 + y1 * y2) + x_r * (x1 - x2) + y_r * (y1 - y2)) / (((x1 - x2) ** 2) + ((y1 - y2) ** 2))
    
    nearest_state[0] = l * waypoints[0, idx_1] + (1 - l) * waypoints[0, idx_2]
    nearest_state[1] = l * waypoints[1, idx_1] + (1 - l) * waypoints[1, idx_2]
    nearest_state[2] = l * waypoints[2, idx_1] + (1 - l) * waypoints[2, idx_2]

    return MX(vcat(nearest_state))

In [91]:
vx_des = 14.0
vy_des = 0.0

s = MX.sym('s', 5)
x, y, yaw, vx, vy = s[0], s[1], s[2], s[3], s[4]

u = MX.sym('u', 2)
acc, delta = [u[0], u[1]]   

prediction = vertcat(x + sqrt(vx**2 + vy**2) * cos(atan2(tan(delta), 2) + yaw) * 0.03,
                        y + sqrt(vx**2 + vy**2) * sin(atan2(tan(delta), 2) + yaw) * 0.03,
                        yaw + (sqrt(vx**2 + vy**2) * tan(delta) * 0.03 / sqrt((19.8025) + (4.95 * tan(delta)**2))),
                        vx + ((4.22 * acc) - (-0.0013 * sqrt(vx**2 + vy**2) * vx - 0.362 * vx)) * 0.03,
                        vy - (1.318 * (atan2(vy, vx) - delta) + (-0.0013 * sqrt(vx**2 + vy**2) + 0.362) * vy) * 0.03)

pred = Function('pred', [s, u], [prediction])

In [92]:
opti = Opti()

N = 1
s = opti.variable(5, N + 1)
e = opti.variable(5, N + 1)
u = opti.variable(2, N)
p = opti.parameter(5, 1)

opti.minimize(sumsqr(s))

for k in range(N):
    opti.subject_to(e[:, k] == s[:, k] - nearest_state(s[:, k], waypoints, vx_des, vy_des))
    opti.subject_to(s[:, k+1] == pred(s[:, k], u[:, k]))

opti.subject_to(s[:, 0] == p)

# Set using true states
opti.set_value(p, [waypoints[1][0],waypoints[1][1],waypoints[1][2], vx_des, vy_des])

opti.subject_to( opti.bounded(0, u[0, :], 1) )
opti.subject_to(opti.bounded(-1.22, u[1, :], 1.22))

# Avoid NaN error
opti.set_initial(s, 1)

opti.solver('ipopt')

sol = opti.solve()

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       44
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:       17

Total number of variables............................:       22
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       15
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        2
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  